# [1280. Students and Examinations](https://leetcode.com/problems/students-and-examinations/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Students

<pre>+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| student_id    | int     |
| student_name  | varchar |
+---------------+---------+</pre>
student_id is the primary key (column with unique values) for this table.
Each row of this table contains the ID and the name of one student in the school.
 

Table: Subjects

<pre>+--------------+---------+
| Column Name  | Type    |
+--------------+---------+
| subject_name | varchar |
+--------------+---------+</pre>
subject_name is the primary key (column with unique values) for this table.
Each row of this table contains the name of one subject in the school.
 

Table: Examinations

<pre>+--------------+---------+
| Column Name  | Type    |
+--------------+---------+
| student_id   | int     |
| subject_name | varchar |
+--------------+---------+</pre>
There is no primary key (column with unique values) for this table. It may contain duplicates.
Each student from the Students table takes every course from the Subjects table.
Each row of this table indicates that a student with ID student_id attended the exam of subject_name.
 

Write a solution to find the number of times each student attended each exam.

Return the result table ordered by student_id and subject_name.

The result format is in the following example.

 

Example 1:

Input: 
Students table:
<pre>+------------+--------------+
| student_id | student_name |
+------------+--------------+
| 1          | Alice        |
| 2          | Bob          |
| 13         | John         |
| 6          | Alex         |
+------------+--------------+</pre>
Subjects table:
<pre>+--------------+
| subject_name |
+--------------+
| Math         |
| Physics      |
| Programming  |
+--------------+</pre>
Examinations table:
<pre>+------------+--------------+
| student_id | subject_name |
+------------+--------------+
| 1          | Math         |
| 1          | Physics      |
| 1          | Programming  |
| 2          | Programming  |
| 1          | Physics      |
| 1          | Math         |
| 13         | Math         |
| 13         | Programming  |
| 13         | Physics      |
| 2          | Math         |
| 1          | Math         |
+------------+--------------+</pre>
Output: 
<pre>+------------+--------------+--------------+----------------+
| student_id | student_name | subject_name | attended_exams |
+------------+--------------+--------------+----------------+
| 1          | Alice        | Math         | 3              |
| 1          | Alice        | Physics      | 2              |
| 1          | Alice        | Programming  | 1              |
| 2          | Bob          | Math         | 1              |
| 2          | Bob          | Physics      | 0              |
| 2          | Bob          | Programming  | 1              |
| 6          | Alex         | Math         | 0              |
| 6          | Alex         | Physics      | 0              |
| 6          | Alex         | Programming  | 0              |
| 13         | John         | Math         | 1              |
| 13         | John         | Physics      | 1              |
| 13         | John         | Programming  | 1              |
+------------+--------------+--------------+----------------+</pre>
Explanation: 
The result table should contain all students and all subjects.
Alice attended the Math exam 3 times, the Physics exam 2 times, and the Programming exam 1 time.
Bob attended the Math exam 1 time, the Programming exam 1 time, and did not attend the Physics exam.
Alex did not attend any exams.
John attended the Math exam 1 time, the Physics exam 1 time, and the Programming exam 1 time.

In [2]:
# Pandas Schema

import pandas as pd

data = [[1, 'Alice'], [2, 'Bob'], [13, 'John'], [6, 'Alex']]
students = pd.DataFrame(data, columns=['student_id', 'student_name']).astype(
    {'student_id': 'Int64', 'student_name': 'object'})
data = [['Math'], ['Physics'], ['Programming']]
subjects = pd.DataFrame(data, columns=['subject_name']).astype({'subject_name': 'object'})
data = [[1, 'Math'], [1, 'Physics'], [1, 'Programming'], [2, 'Programming'], [1, 'Physics'], [1, 'Math'], [13, 'Math'],
        [13, 'Programming'], [13, 'Physics'], [2, 'Math'], [1, 'Math']]
examinations = pd.DataFrame(data, columns=['student_id', 'subject_name']).astype(
    {'student_id': 'Int64', 'subject_name': 'object'})

In [3]:
# Pyspark conversion

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("LocalPySparkNow") \
    .config("spark.driver.memory", "1g") \
    .config("spark.executor.memory", "1g") \
    .config("spark.sql.adaptive.enabled", "false") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "false") \
    .config("spark.ui.enabled", "true") \
    .config("spark.sql.shuffle.partitions", "1") \
    .getOrCreate()

# Uses spark.sparkContext.defaultParallelism = 543. So if the dataset has 15 partitioned in 543 you will get 543 tasks.
students_df = spark.createDataFrame(students).repartition(1)
subjects_df = spark.createDataFrame(subjects).repartition(1)
examinations_df = spark.createDataFrame(examinations).repartition(1)

In [4]:
students_df.show()

+----------+------------+
|student_id|student_name|
+----------+------------+
|         1|       Alice|
|         2|         Bob|
|        13|        John|
|         6|        Alex|
+----------+------------+



In [5]:
subjects_df.show()

+------------+
|subject_name|
+------------+
|        Math|
|     Physics|
| Programming|
+------------+



In [6]:
examinations_df.show()

+----------+------------+
|student_id|subject_name|
+----------+------------+
|         1|        Math|
|         1|     Physics|
|         1| Programming|
|         2| Programming|
|         1|     Physics|
|         1|        Math|
|        13|        Math|
|        13| Programming|
|        13|     Physics|
|         2|        Math|
|         1|        Math|
+----------+------------+



In [ ]:
# In spark Dataframe
from pyspark.sql import functions as F

#Making cartesian product to have all subjects related to all students.



+----------+------------+------------+-----+-----------+
|student_id|student_name|subject_name|st_id|   sub_name|
+----------+------------+------------+-----+-----------+
|         1|       Alice|        Math|    1|       Math|
|         1|       Alice|        Math|    1|       Math|
|         1|       Alice|        Math|    1|       Math|
|         1|       Alice|     Physics|    1|    Physics|
|         1|       Alice|     Physics|    1|    Physics|
|         1|       Alice| Programming|    1|Programming|
|         2|         Bob|        Math|    2|       Math|
|         2|         Bob|     Physics| NULL|       NULL|
|         2|         Bob| Programming|    2|Programming|
|         6|        Alex|        Math| NULL|       NULL|
|         6|        Alex|     Physics| NULL|       NULL|
|         6|        Alex| Programming| NULL|       NULL|
|        13|        John|        Math|   13|       Math|
|        13|        John|     Physics|   13|    Physics|
|        13|        John| Progr

"'student_id','student_name','subject_name','exam_count"

In [17]:

full_df\
      .withColumn('exam_count', F.when(F.isnull('sub_name'),0).otherwise(1))\
      .groupBy('student_id','student_name','subject_name')\
      .agg(F.sum('exam_count').alias('exam_grades'))\
      .select('student_id','student_name','subject_name','exam_grades')\
      .orderBy(F.col('student_id').asc(),F.col('subject_name').asc())\
      .show()


+----------+------------+------------+-----------+
|student_id|student_name|subject_name|exam_grades|
+----------+------------+------------+-----------+
|         1|       Alice|        Math|          3|
|         1|       Alice|     Physics|          2|
|         1|       Alice| Programming|          1|
|         2|         Bob|        Math|          1|
|         2|         Bob|     Physics|          0|
|         2|         Bob| Programming|          1|
|         6|        Alex|        Math|          0|
|         6|        Alex|     Physics|          0|
|         6|        Alex| Programming|          0|
|        13|        John|        Math|          1|
|        13|        John|     Physics|          1|
|        13|        John| Programming|          1|
+----------+------------+------------+-----------+



In [8]:
# In spark SQL

"""
students_df.createOrReplaceTempView('students')
subjects_df.createOrReplaceTempView('subjects')
examinations_df.createOrReplaceTempView('examinations')

spark.sql('''
with students_subjects as (
    select * from students
    join subjects 
)
select 
    students_subjects.student_id,
    students_subjects.student_name,
    students_subjects.subject_name,
    sum(
    case when examinations.student_id is null then 0
    else 1 end
    ) as attended_exams
from students_subjects
left join examinations 
on students_subjects.student_id = examinations.student_id and students_subjects.subject_name = examinations.subject_name
group by students_subjects.student_id, students_subjects.student_name, students_subjects.subject_name
order by student_id, subject_name;
''').show()"""

"\nstudents_df.createOrReplaceTempView('students')\nsubjects_df.createOrReplaceTempView('subjects')\nexaminations_df.createOrReplaceTempView('examinations')\n\nspark.sql('''\nwith students_subjects as (\n    select * from students\n    join subjects \n)\nselect \n    students_subjects.student_id,\n    students_subjects.student_name,\n    students_subjects.subject_name,\n    sum(\n    case when examinations.student_id is null then 0\n    else 1 end\n    ) as attended_exams\nfrom students_subjects\nleft join examinations \non students_subjects.student_id = examinations.student_id and students_subjects.subject_name = examinations.subject_name\ngroup by students_subjects.student_id, students_subjects.student_name, students_subjects.subject_name\norder by student_id, subject_name;\n''').show()"

In [9]:
#spark.stop()